In [1]:
using JSON

In [2]:
"""
    Job

## Attributes
- `index`: j
- `task_sequence`: list of task indices i
- `due_date`: dⱼ
- `weight`: wⱼ
"""
Base.@kwdef struct Job
    index::Int
    task_sequence::Vector{Int}
    release_date::Int
    due_date::Int
    weight::Float64
end

"""
    Task

## Attributes
- `index`: i
- `processing_time`: pᵢ
- `machines`: list of machine indices on which this task can be performed
"""
Base.@kwdef struct Task
    index::Int
    processing_time::Int
    machines::Vector{Int}
end

"""
    Instance

## Attributes
- `nb_operators::Int`
- `α::Float64`: penalty when a job is late
- `β::Float64`: delay penalty for jobs
- `jobs::Vector{Job}`: job list, in index order
- `tasks::Vector{Task}`: task list, in index order
- `operators::Matrix{Vector{Int}}`: operators[i, m] = list of operators that can operate task i on machine m
"""
Base.@kwdef struct Instance
    nb_operators::Int
    α::Float64 # unit penalty
    β::Float64 # tardiness
    jobs::Vector{Job}
    tasks::Vector{Task}
    operators::Matrix{Vector{Int}}  # operators[i, m] = list of operators that can operate task i on machine m
end

nb_jobs(instance::Instance) = length(instance.jobs)
nb_tasks(instance::Instance) = length(instance.tasks)
nb_machines(instance::Instance) = size(instance.operators, 2)
nb_operators(instance::Instance) = instance.nb_operators

nb_operators (generic function with 1 method)

In [3]:
"""
    Solution

## Attributes
- `starts`: starting date of each task.
- `machines`: machines for each task.
- `operators`: operator for each task.
"""
Base.@kwdef struct Solution
    starts::Vector{Int}
    machines::Vector{Int}
    operators::Vector{Int}
end

Solution

In [4]:
"""
    is_feasible(solution::Solution, instance::Instance; verbose=true)

Check if `solution` is feasible for `instance`.
Prints some warnings when solution is infeasible and `verbose` is `true`.
"""
function is_feasible(solution::Solution, instance::Instance; verbose=true)
    (; starts, machines, operators) = solution

    n = nb_tasks(instance)
    @assert n == length(starts) "Not all tasks are in the solution"
    @assert n == length(machines) "Not all tasks are in the solution"
    @assert n == length(operators) "Not all tasks are in the solution"

    # Check job related constraints
    for job in instance.jobs
        current_time = job.release_date

        for task_index in job.task_sequence
            task = instance.tasks[task_index]

            start_time = starts[task_index]
            # start time should occur after end of previous task (constraints 4 and 5)
            if start_time < current_time
                verbose && @warn "Task $task_index started before previous one (or before the release date if it's the first one)"
                return false
            end

            current_time += task.processing_time

            machine_index = machines[task_index]
            # the task needs to be compatible with the chosen machine
            if !(machine_index in task.machines)
                verbose && @warn "Machine $machine_index is not compatible with task $task_index"
                return false
            end

            operator = operators[task_index]
            # the chosen machine should be compatible with the chosen operator
            if ! (operator in instance.operators[task_index, machine_index])
                @warn "Operator $operator cannot operate machine $machine_index"
                return false
            end
        end
    end

    # A machine cannot operate two tasks at the same time (constraint 7)
    for m in 1:nb_machines(instance)
        tasks_with_m = sort(findall(x -> x == m, machines); by=i -> starts[i])

        m_time = 0
        for i in tasks_with_m
            if starts[i] < m_time
                @warn "Two tasks (including $i) at the same time on machine $m"
                return false
            end
            m_time = starts[i] + instance.tasks[i].processing_time
        end
    end

    # An operator cannot operate two tasks at the same time (constraint 7)
    for o in 1:nb_operators(instance)
        tasks_with_o = sort(findall(x -> x == o, operators); by=i -> starts[i])
        o_time = 0
        for i in tasks_with_o
            if starts[i] < o_time
                @warn "Two tasks at the same time with operator $o ($(starts[i]), $o_time)"
                return false
            end
            o_time = starts[i] + instance.tasks[i].processing_time
        end
    end

    return true
end

"""
    job_cost(job_index::Int, solution::Solution, instance::Instance)

Compute the value of job `job_index` in the objective for `solution` in `instance`.
"""
function job_cost(job_index::Int, solution::Solution, instance::Instance)
    (; α, β, jobs, tasks) = instance

    job = jobs[job_index]

    # Compute the job completion time
    last_task_index = job.task_sequence[end]
    last_task = tasks[last_task_index]
    completion_time = solution.starts[last_task_index] + last_task.processing_time

    is_late = completion_time > job.due_date
    tardiness = is_late ? completion_time - job.due_date : 0

    return job.weight * (completion_time + α * is_late + β * tardiness)
end

"""
    cost(solution::Solution, instance::Instance)

Compute the objective value of `solution` for given `instance`.
"""
function cost(solution::Solution, instance::Instance)
    return sum(job_cost(j, solution, instance) for j in 1:nb_jobs(instance))
end

cost

In [5]:
"""
    read_instance(path::String)

Read instance from json file `path`.
"""
function read_instance(path::String)
    @assert endswith(path, ".json")

    data = JSON.parsefile(path)
    parameters_data = data["parameters"]
    nb_tasks = parameters_data["size"]["nb_tasks"]
    nb_machines = parameters_data["size"]["nb_machines"]
    nb_operators = parameters_data["size"]["nb_operators"]
    α = parameters_data["costs"]["unit_penalty"]
    β = parameters_data["costs"]["tardiness"]

    jobs_data = data["jobs"]
    jobs = [
        Job(;
            index=job["job"],
            task_sequence=job["sequence"],
            release_date=job["release_date"],
            due_date=job["due_date"],
            weight=job["weight"],
        ) for job in jobs_data
    ]

    tasks_data = data["tasks"]
    tasks = [
        Task(;
            index=task["task"],
            processing_time=task["processing_time"],
            machines=[machine["machine"] for machine in task["machines"]],
        ) for task in tasks_data
    ]

    operators = [Int[] for _ in 1:nb_tasks, _ in 1:nb_machines]

    for task in tasks_data
        i = task["task"]
        for machine in task["machines"]
            m = machine["machine"]
            operator_list = machine["operators"]
            operators[i, m] = operator_list
        end
    end

    return Instance(; nb_operators, α, β, jobs, tasks, operators)
end

"""
    read_solution(path::String)

Read solution from json file `path`.
"""
function read_solution(path::String)
    @assert endswith(path, ".json")

    data = JSON.parsefile(path)
    nb_tasks = length(data)
    starts = zeros(Int, nb_tasks)
    machines = zeros(Int, nb_tasks)
    operators = zeros(Int, nb_tasks)
    for task in data
        task_index = task["task"]
        starts[task_index] = task["start"]
        machines[task_index] = task["machine"]
        operators[task_index] = task["operator"]
    end
    return Solution(; starts, machines, operators)
end

"""
    write_solution(solution::Solution, path::String)

Write `solution` to file `path` with json format.
"""
function write_solution(solution::Solution, path::String)
    @assert endswith(path, ".json")

    (; starts, machines, operators) = solution
    data = []
    for (task_index, (start, machine, operator)) in
        enumerate(zip(starts, machines, operators))
        push!(
            data,
            Dict(
                "task" => task_index,
                "start" => start,
                "machine" => machine,
                "operator" => operator,
            ),
        )
    end
    open(path, "w") do f
        JSON.print(f, data)
    end
end

"""
    prepare_submission(solver, instances_folder, solutions_folder, group)

Read instances from the `instance_folder`, use `solver` to generate solutions, and then write these solutions in the `solutions_folder` with the right format and `group` number.

The `solver` should be a function with one `Instance` argument, and return a (feasible) `Solution`.
"""
function prepare_submission(;
    solver,
    instances_folder::String="instances",
    solutions_folder::String="solutions",
    group::Int=42,
)
    names = ["KIRO-tiny", "KIRO-medium", "KIRO-large", "KIRO-huge"]
    total_cost = 0.0
    for name in names
        @info "Solving $name instance"
        instance = read_instance(joinpath(instances_folder, "$name.json"))
        sol = solver(instance)  # call the solver
        feasible = is_feasible(sol, instance)
        if feasible
            write_solution(sol, joinpath(solutions_folder, "KIRO-$name-sol_$group.json"))
            total_cost += cost(sol, instance)
        else
            total_cost += Inf
        end
    end
    @info "Total cost: $total_cost"
    return nothing
end

prepare_submission

In [12]:
instance=read_instance("Instances/KIRO-tiny.json")

Instance(8, 6.0, 1.0, Job[Job(1, [1, 9, 10, 15, 16], 2, 8, 6.0), Job(2, [2, 8, 14, 18, 20, 22, 24], 1, 10, 9.0), Job(3, [3, 7, 19], 6, 9, 3.0), Job(4, [4, 6, 11, 12, 13, 17, 21, 23, 25], 5, 19, 14.0), Job(5, [5], 9, 10, 1.0)], Task[Task(1, 1, [1]), Task(2, 1, [6]), Task(3, 1, [4]), Task(4, 2, [2, 7]), Task(5, 1, [2, 6]), Task(6, 2, [1, 3]), Task(7, 1, [5]), Task(8, 1, [4, 7]), Task(9, 1, [6]), Task(10, 1, [8])  …  Task(16, 1, [5]), Task(17, 2, [2, 6, 7]), Task(18, 1, [1, 5]), Task(19, 1, [1, 2, 6, 8]), Task(20, 2, [1]), Task(21, 2, [3, 4, 6]), Task(22, 1, [5, 8]), Task(23, 1, [3, 5, 8]), Task(24, 1, [2, 4]), Task(25, 1, [5, 7, 8])], [[1, 4, 5, 7] Int64[] … Int64[] Int64[]; Int64[] Int64[] … Int64[] Int64[]; … ; Int64[] [2, 6] … Int64[] Int64[]; Int64[] Int64[] … [6, 8] [1, 6, 8]])

In [13]:
sol=read_solution("SOL/glouton_sort_date/KIRO-tiny-sol_11.json")

Solution([3, 2, 7, 7, 10, 9, 8, 3, 4, 5  …  9, 15, 6, 9, 9, 17, 9, 18, 10, 19], [1, 6, 4, 2, 2, 3, 5, 4, 6, 8  …  5, 2, 5, 2, 1, 3, 8, 5, 4, 5], [1, 1, 1, 2, 1, 8, 1, 2, 1, 3  …  2, 5, 3, 1, 6, 8, 3, 7, 2, 4])

In [14]:
t=is_feasible(sol,instance)

true

In [15]:
instance=read_instance("Instances/KIRO-small.json")

Instance(15, 6.0, 1.0, Job[Job(1, [1, 28, 41, 67, 81, 87, 132, 134, 139], 3, 21, 12.0), Job(2, [2, 50, 97, 107, 126, 135, 150], 7, 44, 11.0), Job(3, [3, 26, 33, 34, 37, 39, 63, 95, 115, 127, 131], 10, 93, 17.0), Job(4, [4, 23, 38, 62, 70, 72, 82, 85, 105, 148], 8, 80, 15.0), Job(5, [5, 44, 54, 114], 5, 11, 5.0), Job(6, [6, 58, 73, 89, 103, 111, 140], 9, 34, 8.0), Job(7, [7, 49, 52, 59, 60, 65, 84, 88, 90, 106, 117, 119, 133, 145], 4, 42, 20.0), Job(8, [8, 27, 35, 66, 76, 79, 147], 5, 21, 9.0), Job(9, [9, 31, 32, 47, 51, 64, 75, 83, 108, 125], 7, 42, 15.0), Job(10, [10, 24, 36, 46, 99, 123, 141], 8, 29, 11.0), Job(11, [11, 29, 93, 96, 100, 116, 129, 138, 143], 2, 61, 13.0), Job(12, [12, 55, 74, 78, 94, 136], 9, 17, 8.0), Job(13, [13, 40, 109, 113, 149], 6, 42, 8.0), Job(14, [14, 45, 48, 92, 101, 128], 7, 33, 11.0), Job(15, [15, 22, 25, 56, 61, 68, 69, 104, 118, 122, 144], 1, 63, 19.0), Job(16, [16, 30, 86, 91, 130, 137, 146], 10, 53, 9.0), Job(17, [17, 21, 43, 57, 80, 110], 9, 46, 10.0)

In [16]:
sol=read_solution("SOL/glouton_sort_date/KIRO-small-sol_11.json")

Solution([4, 8, 12, 10, 6, 10, 6, 7, 8, 10  …  25, 26, 20, 26, 28, 25, 19, 27, 19, 25], [3, 4, 4, 4, 2, 5, 4, 1, 5, 6  …  8, 2, 5, 3, 1, 4, 2, 6, 3, 1], [4, 9, 8, 11, 3, 6, 15, 5, 4, 1  …  3, 8, 6, 4, 2, 8, 1, 5, 14, 5])

In [17]:
t=is_feasible(sol,instance)

true

In [18]:
instance=read_instance("Instances/KIRO-medium.json")

Instance(20, 6.0, 1.0, Job[Job(1, [1, 92, 121, 289], 27, 34, 7.0), Job(2, [2, 62, 125, 143, 228, 252, 278], 30, 76, 11.0), Job(3, [3, 97, 163, 196, 217, 270], 7, 25, 8.0), Job(4, [4, 70, 74, 84, 108, 151, 177, 235], 26, 36, 10.0), Job(5, [5, 80, 81, 100, 182, 291], 29, 45, 10.0), Job(6, [6, 101, 157, 162, 164, 283], 10, 30, 7.0), Job(7, [7, 85, 185, 246, 255, 293], 18, 56, 8.0), Job(8, [8, 166, 172, 180, 213, 215, 237, 276, 290, 296], 18, 63, 16.0), Job(9, [9, 89, 113, 120, 130, 169, 173, 178, 241], 3, 21, 15.0), Job(10, [10, 54, 109, 117, 167, 175, 190], 25, 56, 10.0)  …  Job(41, [41, 71, 94, 140, 153, 181, 201], 7, 27, 13.0), Job(42, [42, 106, 259], 26, 32, 3.0), Job(43, [43, 136, 168, 176, 184, 193, 253], 21, 68, 11.0), Job(44, [44, 57, 133, 183, 256, 257], 21, 37, 9.0), Job(45, [45, 55, 64, 138, 159], 25, 59, 7.0), Job(46, [46, 68, 110, 135, 142, 198, 220, 261], 2, 54, 12.0), Job(47, [47, 87, 155, 158, 188, 192, 226, 231], 6, 25, 12.0), Job(48, [48, 53, 124, 202, 219, 299, 300], 1,

In [19]:
sol=read_solution("SOL/glouton_sort_date/KIRO-medium-sol_11.json")

Solution([29, 32, 8, 28, 31, 22, 21, 20, 5, 27  …  39, 32, 27, 36, 8, 36, 23, 14, 10, 25], [3, 6, 7, 12, 16, 1, 2, 4, 7, 11  …  7, 11, 16, 1, 2, 4, 10, 13, 17, 1], [3, 6, 7, 13, 16, 1, 2, 6, 7, 11  …  8, 11, 17, 1, 2, 4, 9, 14, 16, 1])

In [20]:
t=is_feasible(sol,instance)

true

In [23]:
instance_tiny=read_instance("Instances/KIRO-tiny.json")
sol_tiny=read_solution("SOL/glouton_sort_date/KIRO-tiny-sol_11.json")
t_tiny=is_feasible(sol,instance)
instance_small=read_instance("Instances/KIRO-small.json")
sol_small=read_solution("SOL/glouton_sort_date/KIRO-small-sol_11.json")
t_small=is_feasible(sol,instance)
instance_medium=read_instance("Instances/KIRO-medium.json")
sol_medium=read_solution("SOL/glouton_sort_date/KIRO-medium-sol_11.json")
t_medium=is_feasible(sol,instance)
instance_large=read_instance("Instances/KIRO-large.json")
sol_large=read_solution("SOL/glouton_sort_date/KIRO-large-sol_11.json")
t_large=is_feasible(sol,instance)

true

In [25]:
cost(sol_large,instance_large)+cost(sol_tiny,instance_large)+cost(sol_large,instance_large)+cost(sol_large,instance_large)

65041.0